In [29]:
!pip install pydeseq2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 10.6 MB/s eta 0:00:00


In [97]:
import pandas as pd

# Load the CSV file
input_csv = '/Users/sm3662/Downloads/PAC-1/rnasequence.csv'  # replace with your actual file path
df_meta = pd.read_csv(input_csv, encoding='unicode_escape')

# Filter rows where 'CTEP SDCDescription' contains 'breast cancer'
filtered_df = df_meta[df_meta['CTEP SDCDescription'].str.contains('breast', case=False, na=False)]

# Add base URL to 'RSEM(genes)' column and create wget commands
base_url = 'https://pdmdb.cancer.gov/'
wget_commands = filtered_df['RSEM(genes)'].apply(lambda x: f"wget {base_url + x}")

# Save the wget commands to a bash script
bash_script_path = '/Users/sm3662/Downloads/PAC-1/download_rsem_genes.sh'
with open(bash_script_path, 'w') as file:
    file.write('#!/bin/bash\n')
    file.write('\n'.join(wget_commands))

print(f'Bash script with wget commands saved to {bash_script_path}')

Bash script with wget commands saved to /Users/sm3662/Downloads/PAC-1/download_rsem_genes.sh


In [15]:
#Run the bash script, downloaded the files

In [27]:
import os
import pandas as pd

# Path to the folder containing the files
data_folder = '/Users/sm3662/Downloads/PAC-1/Data'

# Initialize an empty dictionary to store data
data_dict = {}

# Iterate through each file in the folder
for filename in os.listdir(data_folder):
    if filename.endswith('.results'):
        file_path = os.path.join(data_folder, filename)
        # Read the file into a DataFrame
        df = pd.read_csv(file_path, sep='\t')
        # Extract gene_id and expected_count
        file_data = df.set_index('gene_id')['expected_count'].to_dict()
        # Add data to the dictionary with the filename (without .result) as the key
        data_dict[filename.replace('.results', '')] = file_data

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(data_dict).T

# Save the DataFrame to a CSV file
result_df.to_csv('/Users/sm3662/Downloads/PAC-1/Gene_Counts.csv')

print("CSV file 'combined_data.csv' has been created successfully.")


CSV file 'combined_data.csv' has been created successfully.


In [71]:
import pandas as pd
import numpy as np
from scipy.stats import gmean

# Path to your file
counts = '/Users/sm3662/Downloads/PAC-1/Gene_Counts.csv'

# Read the file into a DataFrame
df = pd.read_csv(counts, index_col=0)

# Convert all values to numeric, forcing non-numeric to NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Drop any rows or columns that contain NaN values
df.dropna(inplace=True)

# Define the pseudo count value
pseudo_count = 1e-4

# Add the pseudo count to all elements
df += pseudo_count

# Step 1: Transpose the DataFrame
df_transposed = df.T

# Step 2: Create a pseudo-reference sample (row-wise geometric mean)
pseudo_ref = gmean(df_transposed, axis=1)
df_transposed['Pseudo_Ref'] = pseudo_ref

# Step 3: Calculate the ratio of each sample to the reference
ratio_df = df_transposed.div(df_transposed['Pseudo_Ref'], axis=0)
ratio_df = ratio_df.drop(columns=['Pseudo_Ref'])

# Step 4: Calculate the normalization factor for each sample (size factor)
size_factors = ratio_df.median(axis=0)

# Step 5: Calculate the normalized count values using the normalization factor
normalized_df = df_transposed.div(size_factors, axis=1)
normalized_df = normalized_df.T

# Optionally, save the normalized DataFrame to a new file
normalized_file_path = '/Users/sm3662/Downloads/PAC-1/normalized_data.csv'
normalized_df.to_csv(normalized_file_path)

print(f"Normalized data saved to '{normalized_file_path}'.")


Normalized data saved to '/Users/sm3662/Downloads/PAC-1/normalized_data.csv'.


In [157]:
import pandas as pd


# Assuming ABCB1 is a column in df_counts, sort by ABCB1 value
# Replace 'ABCB1' with the actual column name if different
df_sorted = normalized_df.sort_values(by='ABCB1', ascending=False)
df_sorted = df_sorted[['ABCB1']]
# Extract the 'RSEM(genes)' column from df_input for matching
# Assuming 'RSEM(genes)' is the column to be used for matching
rsem_genes_col = 'RSEM(genes)'

# Create a list to store combined rows
combined_rows = []
columns_to_keep = [
    'ABCB1', 'Patient ID', 'Specimen ID', 'Sample ID', 
    'CTEP SDCDescription', 'DiagnosisSubtype', 
    'OncoTreeCode', 'Disease BodyLocation', 'PDM Type'
]

for index, row in df_sorted.iterrows():
    match_row = df_meta[df_meta[rsem_genes_col].str.contains(index.split('genes.')[0])]
    if not match_row.empty:
        # Combine the ABCB1 value with the matched row
        row_data = pd.DataFrame({'ABCB1': [row['ABCB1']], 'Index': [index]})
        combined_row = row_data.join(match_row.reset_index(drop=True))
        combined_rows.append(combined_row)

# Concatenate all combined rows into a single DataFrame
if combined_rows:
    correspondence_df = pd.concat(combined_rows).reset_index(drop=True)
    correspondence_df = correspondence_df[columns_to_keep + ['ABCB1']].iloc[:,1:]
    
    # Optionally, save the correspondence DataFrame to a new file
    correspondence_file_path = '/Users/sm3662/Downloads/PAC-1/correspondence_data.csv'
    correspondence_df.to_csv(correspondence_file_path, index=False)

    print(f"Correspondence data saved to '{correspondence_file_path}'.")
else:
    print("No matching rows found.")


Correspondence data saved to '/Users/sm3662/Downloads/PAC-1/correspondence_data.csv'.
